In [1]:
#4608 training data
import keras
import tensorflow as tf
print('TensorFlow version:', tf.__version__) 
print('Keras version:', keras.__version__)
import sys
import os
from os.path import join
import json
import random
import itertools
import re
import datetime
#import cairocffi as cairo
import editdistance
import numpy as np
from scipy import ndimage
import pylab
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model, load_model
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks
import cv2


Using TensorFlow backend.


TensorFlow version: 1.8.0
Keras version: 2.2.2


In [2]:
sess = tf.Session()
K.set_session(sess)

In [3]:
sys.path.append(os.path.dirname(os.path.realpath('Textgen')) + "/python")
import Textgen as t

# LOAD MODEL 

In [6]:
from keras.models import model_from_json
# load json and create model
fmodel='lr0.02_mom0.9'

json_file = open(fmodel + '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(fmodel + '.h5')
print("Loaded model from disk") 

Loaded model from disk


In [7]:
sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)
# Compile model
loaded_model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd)

# Test on validation images

In [8]:
tiger_test = t.TextImageGenerator('../data/anpr_ocr__test', 'test', 128, 64, 461, 4) # batch size 
tiger_test.build_data()

net_inp = loaded_model.get_layer(name='the_input').input
net_out = loaded_model.get_layer(name='softmax').output

In [11]:
tiger_test.batch_size,tiger_test.img_h,tiger_test.img_w,K

(461,
 64,
 128,
 <module 'keras.backend' from 'C:\\Users\\is95217\\AppData\\Local\\Continuum\\anaconda3\\envs\\tensorflow\\lib\\site-packages\\keras-2.2.2-py3.6.egg\\keras\\backend\\__init__.py'>)

In [17]:
for inp_value, _ in tiger_test.next_batch(K):   
    nbr_example = inp_value['the_input'].shape[0]    
    print('the_input:',np.shape(inp_value['the_input']))
    print('the_labels:',np.shape(inp_value['the_labels']))
    print('input_length:',np.shape(inp_value['input_length']))
    print('label_length:',np.shape(inp_value['label_length']))
    for i  in range(nbr_example):
        print("labels:", inp_value['the_labels'][i])
    break

the_input: (450, 128, 64, 1)
the_labels: (450, 8)
input_length: (450, 1)
label_length: (450, 1)
labels: [ 7.  8. 11. 17.  4.  0.  0. 34.]
labels: [ 7.  8. 11. 17.  4.  0.  0. 34.]
labels: [ 7.  8. 11. 20.  4.  3.  9. 34.]
labels: [ 7.  8. 11. 21.  5.  5.  0. 34.]
labels: [ 7.  8. 11. 23.  7.  2.  3. 34.]
labels: [ 7.  8. 11. 23.  7.  2.  9. 34.]
labels: [ 7.  8. 11. 27.  9.  7.  8. 34.]
labels: [ 7.  8. 27. 11.  6.  1.  1. 34.]
labels: [ 8.  0. 14. 21.  9.  5.  8. 34.]
labels: [ 8.  1. 15. 12.  6.  5.  0. 34.]
labels: [ 3.  4. 29. 15.  2.  4.  0.  9.]
labels: [ 7.  8. 11. 17.  4.  0.  0. 34.]
labels: [ 0.  5. 13. 21.  4.  5.  2. 34.]
labels: [ 0.  6. 17. 32. 14.  1.  7. 34.]
labels: [ 0.  1. 14. 19. 26.  8.  7. 34.]
labels: [ 4.  2. 11. 17. 17.  4.  7. 34.]
labels: [ 1.  0. 31.  1.  5.  4.  2. 34.]
labels: [ 1.  4. 13. 11.  3.  8.  0. 34.]
labels: [ 1.  0. 31.  9.  3.  6.  8. 34.]
labels: [ 5.  4. 14.  2.  3.  6.  5. 34.]
labels: [ 1.  0. 29. 27.  2.  2.  6. 34.]
labels: [ 0.  6. 17. 1

#  Full test set

In [15]:
tiger_test = t.TextImageGenerator('../data/anpr_ocr__test/img', 'test', 128, 64, 450, 4) # batch size 
tiger_test.build_data()

In [16]:
for inp_value, _ in tiger_test.next_batch(K):
    matched=0
    non_matched=0
    testset_cnt = inp_value['the_input'].shape[0]
    X_data = inp_value['the_input']
    
    net_out_value = sess.run(net_out, feed_dict={net_inp:X_data})
    pred_texts = t.decode_batch(net_out_value)
    
    labels = inp_value['the_labels']
    texts = []     
    
    for label in labels:
        text = t.labels_to_text(label)        
        texts.append(text)       
    
    for cnt in range(testset_cnt):        
        
        if pred_texts[cnt] == texts[cnt]:
            matched += 1            
        else:
            non_matched +=  1
            print('wrong Predicted: %s True: %s i: %s len_pred: %s len_true: %s' % (pred_texts[cnt], texts[cnt],str(cnt),
                                                                              len(pred_texts[cnt]),len(texts[cnt])))
            
    print("matched=",matched, ",total=",testset_cnt , "TP: %", round(matched/testset_cnt, 2) * 100)
    break
    

wrong Predicted: 01DJK52 True: 01DJM52 i: 6 len_pred: 7 len_true: 7
wrong Predicted: 06DK5309 True: 06DK5303 i: 24 len_pred: 8 len_true: 8
wrong Predicted: 06DK5307 True: 06DK5907 i: 29 len_pred: 8 len_true: 8
wrong Predicted: 10Y5859 True: 10Y5869 i: 37 len_pred: 7 len_true: 7
wrong Predicted: 10Y61521 True: 10Y6152 i: 38 len_pred: 8 len_true: 7
wrong Predicted: 34TN1268 True: 13TN1268 i: 40 len_pred: 8 len_true: 8
wrong Predicted: 34F8714 True: 34HF8714 i: 45 len_pred: 7 len_true: 8
wrong Predicted: 4PN428 True: 41PN428 i: 48 len_pred: 6 len_true: 7
wrong Predicted: 41V1146 True: 41V1148 i: 51 len_pred: 7 len_true: 7
wrong Predicted: 411V1149 True: 41V1149 i: 52 len_pred: 8 len_true: 7
wrong Predicted: 41V690 True: 41VN690 i: 56 len_pred: 6 len_true: 7
wrong Predicted: 43HP98 True: 43HP983 i: 74 len_pred: 6 len_true: 7
wrong Predicted: 81EN209 True: 81EH209 i: 82 len_pred: 7 len_true: 7
wrong Predicted: 01DFA25 True: 01DFA251 i: 87 len_pred: 7 len_true: 8
wrong Predicted: 06BHS46 Tru

In [ ]:
# evaluate the model
"""    
inputs = {
            'the_input': X_data,
            'the_labels': Y_data,
            'input_length': input_length,
            'label_length': label_length,
            #'source_str': source_str
        }
#outputs = {'ctc': np.zeros([self.batch_size])}

XX = [X['the_input'], 
      X['the_labels'], 
      X['input_length'], 
      X['label_length']]
"""

#model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

scores = loaded_model.evaluate()
for i in range(len(scores)):
    print("\n%s: %.2f%%" % (loaded_model.metrics_names[i], scores[i]*100))



In [ ]:
X=tiger_test.next_batch(K)
X['the_input']